<a href="https://colab.research.google.com/github/lucas26xd/registrosPRF/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados abertos sobre Acidentes da Polícia Rodoviária Federal a partir da aplicação do processo OSEMN.

# Etapa de Limpeza

In [ ]:
import pandas as pd
from plotly import __version__
if float(__version__[:__version__.rfind('.')]) < 4.11:
  !pip install -U plotly
import plotly.express as px
import seaborn as sn
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/lucas26xd/registrosPRF/master/Bases'
bases = {'Ocorrencias': {'folder': 'Agrupados%20por%20ocorrencia', 
                         'files': ['datatran2017.csv', 'datatran2018.csv', 'datatran2019.csv', 'datatran2020.csv']}, 
         'Pessoas': {'folder': 'Agrupados%20por%20pessoa', 
                     'files': ['acidentes2017.csv', 'acidentes2018.csv', 'acidentes2019.csv', 'acidentes2020.csv']}, 
         }

def import_base(base):
  dfs = list()
  for file in bases[base]['files']:
    dfs.append(pd.read_csv(f'{url}/{bases[base]["folder"]}/{file}', encoding='ISO-8859-1', sep=';'))

  return pd.concat(dfs)

def converte_srt_to_float(df, atrib):
  for i in range(0, len(atrib)):
    df[atrib[i]] = df[atrib[i]].apply(lambda x: x.replace(",", "."))
    df[atrib[i]] = df[atrib[i]].astype(float)

def converte_float_to_int(df, atrib):
  for i in range(0, len(atrib)):
    df[atrib[i]] = df[atrib[i]].astype(int)

### Base de Ocorrêncas

In [ ]:
# UNINDO AS 4 ÚLTIMAS BASES DE OCORRÊNCIAS (2017 a 2020)
dfOcorrencias = import_base('Ocorrencias')
dfOcorrencias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260388 entries, 0 to 34083
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      260388 non-null  float64
 1   data_inversa            260388 non-null  object 
 2   dia_semana              260388 non-null  object 
 3   horario                 260388 non-null  object 
 4   uf                      260388 non-null  object 
 5   br                      259955 non-null  float64
 6   km                      259955 non-null  object 
 7   municipio               260388 non-null  object 
 8   causa_acidente          260388 non-null  object 
 9   tipo_acidente           260388 non-null  object 
 10  classificacao_acidente  260388 non-null  object 
 11  fase_dia                260388 non-null  object 
 12  sentido_via             260388 non-null  object 
 13  condicao_metereologica  260388 non-null  object 
 14  tipo_pista           

In [ ]:
# LIMPEZA BASE OCORRÊNCAS

## Remoção de atributos considerados irrelevantes
dfOcorrencias = dfOcorrencias.drop(columns=['fase_dia', 'sentido_via', 'uso_solo', 'latitude', 'longitude', 'regional', 'delegacia', 'uop'])

## Remove regitros NaN nos atributos br e km
t = len(dfOcorrencias.id)
dfOcorrencias = dfOcorrencias[dfOcorrencias['br'].notna()]
dfOcorrencias = dfOcorrencias[dfOcorrencias['km'].notna()]
print(f'Removidos {t - len(dfOcorrencias.id)} registros.')

## Conversões
atributos_str = ["data_inversa", "dia_semana", "horario", "uf", "municipio", "causa_acidente","tipo_acidente", "classificacao_acidente", "condicao_metereologica", "tipo_pista", "tracado_via"]
atributos_int = ["id", "br", "pessoas", "mortos", "feridos_leves", "feridos_graves", "ilesos", "ignorados", "feridos", "veiculos"]
converte_float_to_int(dfOcorrencias, atributos_int)
converte_srt_to_float(dfOcorrencias, ["km"])

dfOcorrencias['data_inversa'] = pd.to_datetime(dfOcorrencias['data_inversa'])
dfOcorrencias['horario'] = pd.to_datetime(dfOcorrencias['horario'])

dfOcorrencias.info()
dfOcorrencias.head()

Removidos 433 registros.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 259955 entries, 0 to 34083
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      259955 non-null  int64         
 1   data_inversa            259955 non-null  datetime64[ns]
 2   dia_semana              259955 non-null  object        
 3   horario                 259955 non-null  datetime64[ns]
 4   uf                      259955 non-null  object        
 5   br                      259955 non-null  int64         
 6   km                      259955 non-null  float64       
 7   municipio               259955 non-null  object        
 8   causa_acidente          259955 non-null  object        
 9   tipo_acidente           259955 non-null  object        
 10  classificacao_acidente  259955 non-null  object        
 11  condicao_metereologica  259955 non-null  object        
 12  tipo_p

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,condicao_metereologica,tipo_pista,tracado_via,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,8,2017-01-01,domingo,2020-10-06 00:00:00,PR,376,112.0,PARANAVAI,Fenômenos da Natureza,Queda de ocupante de veículo,Com Vítimas Feridas,Chuva,Simples,Reta,1,0,0,1,0,0,1,1
1,9,2017-01-01,domingo,2020-10-06 00:01:00,SC,101,234.0,PALHOCA,Falta de Atenção à Condução,Colisão com objeto estático,Sem Vítimas,Chuva,Dupla,Curva,1,0,0,0,1,0,0,1
2,11,2017-01-01,domingo,2020-10-06 00:00:00,PR,153,56.9,SANTO ANTONIO DA PLATINA,Animais na Pista,Capotamento,Com Vítimas Feridas,Garoa/Chuvisco,Simples,Reta,2,0,2,0,0,0,2,1
3,12,2017-01-01,domingo,2020-10-06 00:00:00,GO,153,435.0,ANAPOLIS,Avarias e/ou desgaste excessivo no pneu,Tombamento,Com Vítimas Feridas,Céu Claro,Dupla,Reta,1,0,0,1,0,0,1,1
4,13,2017-01-01,domingo,2020-10-06 00:00:00,SC,280,77.3,CORUPA,Ingestão de Álcool,Saída de leito carroçável,Com Vítimas Feridas,Chuva,Simples,Não Informado,1,0,1,0,0,0,1,1


### Base de Pessoas

In [ ]:
# UNINDO AS 4 ÚLTIMAS BASES DE PESSOAS (2017 a 2020)
dfPessoas = import_base('Pessoas')
dfPessoas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609882 entries, 0 to 78411
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      609882 non-null  float64
 1   pesid                   609878 non-null  float64
 2   data_inversa            609882 non-null  object 
 3   dia_semana              609882 non-null  object 
 4   horario                 609882 non-null  object 
 5   uf                      609882 non-null  object 
 6   br                      608891 non-null  float64
 7   km                      608891 non-null  object 
 8   municipio               609882 non-null  object 
 9   causa_acidente          609882 non-null  object 
 10  tipo_acidente           609882 non-null  object 
 11  classificacao_acidente  609882 non-null  object 
 12  fase_dia                609882 non-null  object 
 13  sentido_via             609882 non-null  object 
 14  condicao_metereologic

In [ ]:
# LIMPEZA BASE PESSOAS

## Remoção de atributos considerados irrelevantes
dfPessoas = dfPessoas.drop(columns=['fase_dia', 'sentido_via', 'uso_solo', 'id_veiculo', 'marca', 'ano_fabricacao_veiculo', 'latitude', 'longitude', 'regional', 'delegacia', 'uop'])

## Remove regitros NaN nos atributos br e km
t = len(dfPessoas.id)
dfPessoas = dfPessoas[dfPessoas['br'].notna()]
dfPessoas = dfPessoas[dfPessoas['km'].notna()]
print(f'Removidos {t - len(dfPessoas.id)} registros.')

## Pessoas com sexo = 'Ignorado' virarão sexo = 'Não Informado'
dfPessoas.loc[dfPessoas['sexo'] == 'Ignorado', 'sexo'] = 'Não Informado'

## Conversões
dfPessoas['id'] = dfPessoas['id'].astype(int)
dfPessoas['data_inversa'] = pd.to_datetime(dfPessoas['data_inversa'])
dfPessoas['horario'] = pd.to_datetime(dfPessoas['horario'])

### Substituindo valores NaN por PESIDs novos
qtd_pesid_nan = len(dfPessoas[dfPessoas['pesid'].isna()]['pesid'])
dfPessoas.loc[dfPessoas['pesid'].isna(), 'pesid'] = [dfPessoas['pesid'].max() + 1 + i for i in range(qtd_pesid_nan)]
dfPessoas['pesid'] = dfPessoas['pesid'].astype(int)

### Preenchendo idades faltantes ou outliers com a média dos valores válidos
media_idade = dfPessoas.loc[dfPessoas['idade'] <= 110, 'idade'].mean()
dfPessoas.loc[(dfPessoas['idade'].isna()) | (dfPessoas['idade'] > 110), 'idade'] = media_idade
dfPessoas['idade'] = dfPessoas['idade'].astype(int)

dfPessoas.info()
dfPessoas.head()

Removidos 991 registros.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 608891 entries, 0 to 78411
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      608891 non-null  int64         
 1   pesid                   608891 non-null  int64         
 2   data_inversa            608891 non-null  datetime64[ns]
 3   dia_semana              608891 non-null  object        
 4   horario                 608891 non-null  datetime64[ns]
 5   uf                      608891 non-null  object        
 6   br                      608891 non-null  float64       
 7   km                      608891 non-null  object        
 8   municipio               608891 non-null  object        
 9   causa_acidente          608891 non-null  object        
 10  tipo_acidente           608891 non-null  object        
 11  classificacao_acidente  608891 non-null  object        
 12  condic

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,condicao_metereologica,tipo_pista,tracado_via,tipo_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos
0,8,1,2017-01-01,domingo,2020-10-06 00:00:00,PR,376.0,112,PARANAVAI,Fenômenos da Natureza,Queda de ocupante de veículo,Com Vítimas Feridas,Chuva,Simples,Reta,Motocicleta,Condutor,Lesões Graves,19,Masculino,0,0,1,0
1,9,955,2017-01-01,domingo,2020-10-06 00:01:00,SC,101.0,234,PALHOCA,Falta de Atenção à Condução,Colisão com objeto estático,Sem Vítimas,Chuva,Dupla,Curva,Automóvel,Condutor,Ileso,35,Masculino,1,0,0,0
2,11,3,2017-01-01,domingo,2020-10-06 00:00:00,PR,153.0,"56,9",SANTO ANTONIO DA PLATINA,Animais na Pista,Capotamento,Com Vítimas Feridas,Garoa/Chuvisco,Simples,Reta,Automóvel,Passageiro,Lesões Leves,27,Masculino,0,1,0,0
3,11,2,2017-01-01,domingo,2020-10-06 00:00:00,PR,153.0,"56,9",SANTO ANTONIO DA PLATINA,Animais na Pista,Capotamento,Com Vítimas Feridas,Garoa/Chuvisco,Simples,Reta,Automóvel,Condutor,Lesões Leves,27,Feminino,0,1,0,0
4,12,1499,2017-01-01,domingo,2020-10-06 00:00:00,GO,153.0,435,ANAPOLIS,Avarias e/ou desgaste excessivo no pneu,Tombamento,Com Vítimas Feridas,Céu Claro,Dupla,Reta,Motocicleta,Condutor,Lesões Graves,24,Masculino,0,0,1,0


# Etapa Modelagem

### **Hipóteses Lançadas**

---

- Base Pessoas :
  - Número de Acidentes por Faixa Etária
  - Número de Pessoas por Tipo de Acidente
  - Número de Acidentes por Sexo
- Base Ocorrências:
  - Número de Acidentes Totais 
  - Número de Acidentes por Tipo de Veículo
  - Número de Acidentes por Causa de Acidente

## Funções Regressão

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import plotly.graph_objects as go

def hold_out(X, Y, size=0.3):  # Separação da base em treino em teste usando a técnica holdout
  return train_test_split(X, Y, test_size=size)

def label_encoder(df, attrs):
  LE = LabelEncoder()
  for i in attrs:
    df[i] = LE.fit_transform(df[i])
  return df

def standard_scaler(X, Y):  # Escalonamento padrão de toda base
  SS = StandardScaler()
  X = SS.fit_transform(X)
  Y = SS.fit_transform(Y)
  return X, Y

def regressao(regressor, X_train, Y_train, X_test, Y_test):  # Treina o regressor, imprime e retorna suas métricas
  regressor.fit(X_train, Y_train)
  
  score_train = regressor.score(X_train, Y_train)
  score_test = regressor.score(X_test, Y_test)
  print('Score Treino: ', score_train, '\nScore Teste: ', score_test)

  previsoes = regressor.predict(X_test)
  mae = mean_absolute_error(Y_test, previsoes)
  print('Erro médio absoluto: ', mae, '\n')

  return score_train, score_test, mae

def regressao_linear(X_train, Y_train, X_test, Y_test):  # Cria o modelo de regressão linear
  regressor = LinearRegression()
  return regressao(regressor, X_train, Y_train, X_test, Y_test)

def regressao_polinomial(X_train, Y_train, X_test, Y_test, dg = 3):  # Cria o modelo de regressão polinomial
  poly = PolynomialFeatures(degree=dg)
  X_poly_train = poly.fit_transform(X_train)
  X_poly_test = poly.fit_transform(X_test)

  return regressao_linear(X_poly_train, Y_train, X_poly_test, Y_test)

def arvore_decisao(X_train, Y_train, X_test, Y_test):  # Cria o modelo de árvore de decisão
  regressor = DecisionTreeRegressor()
  return regressao(regressor, X_train, Y_train, X_test, Y_test)

def floresta_randomica(X_train, Y_train, X_test, Y_test, n=10):  # Cria o modelo de floresta randômica
  regressor = RandomForestRegressor(n_estimators=n)
  return regressao(regressor, X_train, Y_train, X_test, Y_test)

def mlp(X_train, Y_train, X_test, Y_test, hls=(100,)):  # Cria o modelo de MLP 
  regressor = MLPRegressor(hidden_layer_sizes=hls)
  return regressao(regressor, X_train, Y_train, X_test, Y_test)

def knr(X_train, Y_train, X_test, Y_test, k=2):  # Cria o modelo de regressão dos k vizinhos 
  regressor = KNeighborsRegressor(n_neighbors=k)
  return regressao(regressor, X_train, Y_train, X_test, Y_test)

def plot_scores_bar(regressors, scs1, scs2, title):  # Plota o gráfico de barras dos scores de cada regressor
  fig = go.Figure(data=[go.Bar(name='Scores de Treino', x=regressors, y=scs1), go.Bar(name='Scores de Teste', x=regressors, y=scs2)])
  fig.update_layout(barmode='group', title_text=title)
  fig.show()

## **Base Pessoas**
---

#### **Hipótese 1:**

##### Agrupando Dados

In [ ]:
x = ['ano', 'mes', 'faixa_idade']
y = ['qtd_acidentes', 'qtd_pessoas', 'qtd_ilesos', 'qtd_feridos_leves', 'qtd_feridos_graves', 'qtd_mortos']

df = dfPessoas.copy()

df['idade'].loc[df['idade'] < 24] = 1
df['idade'].loc[df['idade'] > 60] = 3
df['idade'].loc[df['idade'] >= 24] = 2
df['idade'].loc[df['idade'] == 1] = 'Menor de 24 anos' 
df['idade'].loc[df['idade'] == 3] = 'Maior de 60 anos' 
df['idade'].loc[df['idade'] == 2] = 'Entre 24 e 60 anos'

df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%B'), 'idade'])
df = df.agg({'id': 'nunique', 'pesid': 'nunique', 'ilesos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'mortos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A v

,ano,mes,faixa_idade,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2017,April,Entre 24 e 60 anos,7226,14043,7459,4088,1153,381
1,2017,April,Maior de 60 anos,1045,1274,729,388,104,53
2,2017,April,Menor de 24 anos,1782,2586,971,1190,344,81
3,2017,August,Entre 24 e 60 anos,6584,12369,6315,3604,1222,346
4,2017,August,Maior de 60 anos,957,1116,583,375,93,65


##### Tratando Dados

In [ ]:
# Label Encoder
df_pessoas_tratado = label_encoder(df.copy(), ['mes','faixa_idade'])

# Divisão da base
X = df_pessoas_tratado[x]
Y = df_pessoas_tratado[y]

# Standart Scaler
# X, Y = standard_scaler(X, Y)

X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Regressão Linear

In [ ]:
sc1_rl, sc2_rl, mae_rl = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.6308526806555491 
Score Teste:  0.6598230108948874
Erro médio absoluto:  959.1271245710568 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão Polinomial

In [ ]:
sc1_rp, sc2_rp, mae_rp = regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9860956153797502 
Score Teste:  0.9763372673095564
Erro médio absoluto:  203.06904615092483 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Árvore de Decisão

In [ ]:
sc1_ad, sc2_ad, mae_ad = arvore_decisao(X_train, Y_train, X_test, Y_test)

Score Treino:  1.0 
Score Teste:  0.9607071359997094
Erro médio absoluto:  221.08974358974353 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Random Forest

In [ ]:
sc1_rf, sc2_rf, mae_rf = floresta_randomica(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9924883766709951 
Score Teste:  0.979666890877125
Erro médio absoluto:  172.64102564102564 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por MLP

In [ ]:
sc1_rn, sc2_rn, mae_rn = mlp(X_train, Y_train, X_test, Y_test, hls=(200, 100))

Score Treino:  -0.001369014116730468 
Score Teste:  -0.046380150793003734
Erro médio absoluto:  1701.8168619908486 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por KNR

In [ ]:
sc1_kn, sc2_kn, mae_kn = knr(X_train, Y_train, X_test, Y_test, k=5)

Score Treino:  0.7503060603186378 
Score Teste:  0.2716820112751806
Erro médio absoluto:  1204.6914529914532 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



#####Resultados

In [ ]:
regressores = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino = [sc1_rl, sc1_rp, sc1_ad, sc1_rf, sc1_rn, sc1_kn]
scores_teste = [sc2_rl, sc2_rp, sc2_ad, sc2_rf, sc2_rn, sc2_kn]
maes = [mae_rl, mae_rp, mae_ad, mae_rf, mae_rn, mae_kn]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores de TREINO/TESTE dos regressores na base de pessoas agrupadas (Mês, Ano, Faixa Etária).')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de pessoas agrupadas (Mês, Ano, Faixa Etária).', labels={'x': 'Regressores', 'y': 'MAE'})

##### Extrapolação dos dados

###### Criando as novas amostras (X)

In [ ]:
import numpy as np
monthNames = ['July', 'August', 'September', 'October', 'November', 'December'];
ano = ['2020']
faixa_idade = df['faixa_idade'].unique()

comb  = np.array(np.meshgrid(ano, monthNames, faixa_idade)).T.reshape(-1,3)
df_novo1 = pd.DataFrame(comb, columns=['ano', 'mes', 'faixa_idade'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(ano, monthNames, faixa_idade)).T.reshape(-1,3)
df_novo2 = pd.DataFrame(comb, columns=['ano', 'mes', 'faixa_idade'])

df_novo = pd.concat([df_novo1, df_novo2])
df_novo.index = [i for i in range(len(df_novo))]

df_novo_concatenado = pd.concat([df, df_novo])

LE = LabelEncoder()
df_novo_concatenado['faixa_idade'] = LE.fit_transform(df_novo_concatenado['faixa_idade'])
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concatenado.loc[df_novo_concatenado.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concatenado.index = [i for i in range(len(df_novo_concatenado))]

###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concatenado[:len(df)][x]
X_test = df_novo_concatenado[len(df):][x]
Y_train = df[y]

######Random Forest

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(X_train, Y_train)

previsoes = regressor.predict(X_test)

previsoes = pd.DataFrame(previsoes)
previsoes.columns = y

previsoes = pd.concat([df_novo, previsoes], axis=1)
for mes in meses:
  previsoes.loc[previsoes.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes

,ano,mes,faixa_idade,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2020,7,Entre 24 e 60 anos,4648.63,8926.07,3941.28,2952.38,914.37,316.42
1,2020,8,Entre 24 e 60 anos,4647.65,8897.53,3917.53,2953.64,910.50,314.60
2,2020,9,Entre 24 e 60 anos,4659.87,8910.80,3921.65,2962.22,910.75,313.83
3,2020,10,Entre 24 e 60 anos,4659.25,8920.50,3926.62,2963.27,918.08,313.04
4,2020,11,Entre 24 e 60 anos,4655.70,8915.35,3930.87,2956.82,911.93,314.74
5,2020,12,Entre 24 e 60 anos,4754.68,9166.35,4047.81,3049.50,942.45,317.70
6,2020,7,Maior de 60 anos,648.99,738.28,346.16,252.00,95.23,44.89
7,2020,8,Maior de 60 anos,648.06,737.81,345.42,252.01,95.12,45.26
8,2020,9,Maior de 60 anos,647.92,737.07,344.96,252.17,94.86,45.08
9,2020,10,Maior de 60 anos,648.12,737.27,345.18,252.17,95.02,44.90


##### Plot Extrapolação

In [ ]:
def cria_data_plot(attr):
  data_plot = list()
  for faixa_idade in df_plot.faixa_idade.unique():
    data = df_plot[df_plot.faixa_idade == faixa_idade]
    data_plot.append(go.Line(name=(faixa_idade + '- Real'), x=data.mes.values, y=data[attr].values))

  for faixa_idade in previsoes.faixa_idade.unique():
    data = previsoes[(previsoes.faixa_idade == faixa_idade) & (previsoes.ano == '2020')]
    data_plot.append(go.Line(name=(faixa_idade + '- Previsto'), x=data.mes.values, y=data[attr].values))
  return data_plot


In [ ]:
df_novo_concatenado['faixa_idade'] = LE.inverse_transform(df_novo_concatenado['faixa_idade'])

df_plot = df_novo_concatenado[:len(df)][df_novo_concatenado[:len(df)]['ano'] == '2020']
df_plot = df_plot.sort_values('mes')

###### Quantidade de Acidentes

In [ ]:
data_plot = cria_data_plot('qtd_acidentes')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de acidentes por mês em 2020 em cada Faixa Etária - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Pessoas

In [ ]:
data_plot = cria_data_plot('qtd_pessoas')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Pessoas por mês em 2020 em cada Faixa Etária - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Ilesos

In [ ]:
data_plot = cria_data_plot('qtd_ilesos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Ilesos por mês em 2020 em cada Faixa Etária - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Feridos Leves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_leves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Feridos Leves por mês em 2020 em cada Faixa Etária - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Feridos Graves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_graves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Graves Leves por mês em 2020 em cada Faixa Etária - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Mortos

In [ ]:
data_plot = cria_data_plot('qtd_mortos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Mortos Leves por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




#### **Hipótese 2:**

##### Agrupando Dados

In [ ]:
x = ['ano', 'mes', 'tipo_acidente']
y = ['qtd_acidentes', 'qtd_ilesos', 'qtd_feridos_leves', 'qtd_feridos_graves', 'qtd_mortos', 'qtd_feridos', 'qtd_veiculos']

df = dfOcorrencias
df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%B'), 'tipo_acidente'])
df = df.agg({'id': 'nunique', 'ilesos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'mortos': 'sum', 'feridos': 'sum', 'veiculos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

##### Tratando Dados

In [ ]:
# Label Encoder
df_encoded = label_encoder(df.copy(), ['mes', 'tipo_acidente'])

# Divisão da base
X = df_encoded[x]
Y = df_encoded[y]

X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Regressão Linear

In [ ]:
scrl1_tipo, scrl2_tipo, maerl_tipo = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.6566366551817503 
Score Teste:  0.5960782090935575
Erro médio absoluto:  1009.2446363510967 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão Polinomial

In [ ]:
scrl1_tipo, scrl2_tipo, maerp_tipo= regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9872060466297599 
Score Teste:  0.9773227438823342
Erro médio absoluto:  191.63490647536057 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Árvore de Decisão

In [ ]:
scad1_tipo, scad2_tipo, maead_tipo = arvore_decisao(X_train, Y_train, X_test, Y_test)

Score Treino:  1.0 
Score Teste:  0.9669257539790482
Erro médio absoluto:  218.77350427350427 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Random Forest

In [ ]:
scrf1_tipo, scrf2_tipo, maerf_tipo = floresta_randomica(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9967650909926473 
Score Teste:  0.9736071129534679
Erro médio absoluto:  183.92521367521374 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por MLP

In [ ]:
scmpl1_tipo, scmpl2_tipo, maempl_tipo = mlp(X_train, Y_train, X_test, Y_test, hls=(200, 100))

Score Treino:  -0.000692652117398385 
Score Teste:  -0.00579659727338131
Erro médio absoluto:  1607.6859614936932 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por KNR

In [ ]:
sckn1_tipo, sckn2_tipo, maekn_tipo = knr(X_train, Y_train, X_test, Y_test, k=7)

Score Treino:  0.713808521272756 
Score Teste:  0.48163165650740086
Erro médio absoluto:  913.4512820512822 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



#####Resultados

In [ ]:
regressores_tipo = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino_tipo = [scrl1_tipo, scrl1_tipo, scad1_tipo, scrf1_tipo, scmpl1_tipo, sckn1_tipo]
scores_teste_tipo = [scrl2_tipo, scrl2_tipo, scad2_tipo, scrf2_tipo, scmpl2_tipo, sckn2_tipo]
maes_tipo = [maerl_tipo, maerp_tipo, maead_tipo , maerf_tipo , maempl_tipo, maekn_tipo ]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores de TREINO/TESTE dos regressores na base de pessoas agrupadas (Mês, Ano, Faixa Etária).')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de pessoas agrupadas (Mês, Ano, Faixa Etária).', labels={'x': 'Regressores', 'y': 'MAE'})

##### Extrapolação dos Dados

###### Criando novas amostras

In [ ]:
import numpy as np
monthNames = ["August", "September", "October", "November", "December"];
ano = ['2020']
causas = df['tipo_acidente'].unique()

comb  = np.array(np.meshgrid(monthNames,ano,causas)).T.reshape(-1,3)
df_novo1 = pd.DataFrame(comb, columns = ['mes','ano','tipo_acidente'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(monthNames,ano,causas)).T.reshape(-1,3)
df_novo2 = pd.DataFrame(comb, columns = ['mes','ano','tipo_acidente'])

df_novo = pd.concat([df_novo1, df_novo2], axis=0)
df_novo.index = [i for i in range(len(df_novo))]

df_novo_concat = pd.concat([df, df_novo])

LE = LabelEncoder()
df_novo_concat['tipo_acidente'] = LE.fit_transform(df_novo_concat['tipo_acidente'])
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concat.loc[df_novo_concat.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concat.index = [i for i in range(len(df_novo_concat))]
df_novo_concat

,ano,mes,tipo_acidente,qtd_acidentes,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos,qtd_feridos,qtd_veiculos
0,2017,4,0,189.0,184.0,77.0,26.0,8.0,103.0,199.0
1,2017,4,1,310.0,241.0,186.0,124.0,86.0,310.0,340.0
2,2017,4,2,485.0,367.0,352.0,114.0,26.0,466.0,514.0
3,2017,4,3,57.0,88.0,40.0,14.0,4.0,54.0,103.0
4,2017,4,4,624.0,561.0,315.0,63.0,28.0,378.0,700.0
...,...,...,...,...,...,...,...,...,...,...
779,2021,1,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,2021,1,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,2021,1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,2021,1,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concat[:len(df)][x]
X_test = df_novo_concat[len(df):][x]
Y_train = df[y]

###### Arvore de Decisão

In [ ]:
regressor1 = DecisionTreeRegressor()
  
regressor1.fit(X_train, Y_train)
previsoes1 = regressor1.predict(X_test)

previsoes1 = pd.DataFrame(previsoes1)
previsoes1.columns = y

previsoes1 = pd.concat([df_novo, previsoes1], axis=1)
for mes in meses:
  previsoes1.loc[previsoes1.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes1

,mes,ano,tipo_acidente,qtd_acidentes,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos,qtd_feridos,qtd_veiculos
0,8,2020,Atropelamento de Animal,99.0,60.0,72.0,24.0,6.0,96.0,113.0
1,9,2020,Atropelamento de Animal,99.0,54.0,102.0,28.0,3.0,130.0,114.0
2,10,2020,Atropelamento de Animal,109.0,38.0,80.0,37.0,3.0,117.0,116.0
3,11,2020,Atropelamento de Animal,101.0,70.0,78.0,21.0,11.0,99.0,111.0
4,12,2020,Atropelamento de Animal,106.0,74.0,75.0,27.0,9.0,102.0,114.0
...,...,...,...,...,...,...,...,...,...,...
91,1,2021,Engavetamento,100.0,379.0,117.0,16.0,1.0,133.0,392.0
92,1,2021,Incêndio,79.0,87.0,3.0,2.0,0.0,5.0,81.0
93,1,2021,Queda de ocupante de veículo,237.0,36.0,244.0,43.0,6.0,287.0,260.0
94,1,2021,Saída de leito carroçável,988.0,585.0,1152.0,206.0,57.0,1358.0,1036.0


#### **Hipótese 3:**

##### Agrupamento da base Pessoas

In [ ]:
x = ['ano', 'mes', 'sexo']
y = ['qtd_acidentes', 'qtd_pessoas', 'qtd_ilesos', 'qtd_feridos_leves', 'qtd_feridos_graves', 'qtd_mortos']

df = dfPessoas
df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%B'), 'sexo'])
df = df.agg({'id': 'nunique', 'pesid': 'nunique', 'ilesos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'mortos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

df.head()

,ano,mes,sexo,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2017,April,Feminino,2581,3939,1540,1908,394,97
1,2017,April,Masculino,7223,12719,7359,3745,1204,411
2,2017,April,Não Informado,975,1245,260,13,3,7
3,2017,August,Feminino,2307,3317,1252,1574,397,94
4,2017,August,Masculino,6614,11433,6258,3547,1224,404


##### Tratamentos

In [ ]:
# Label Encoder
df_pessoas_tratado = label_encoder(df.copy(), ['mes', 'sexo'])

# Divisão da base
X = df_pessoas_tratado[x]
Y = df_pessoas_tratado[y]

# Standart Scaler
# X, Y = standard_scaler(X, Y)

X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Regressão Linear

In [ ]:
sc1_rl, sc2_rl, mae_rl = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.14699108603950742 
Score Teste:  -0.396483410620267
Erro médio absoluto:  1630.6741219469368 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão Polinomial

In [ ]:
sc1_rp, sc2_rp, mae_rp = regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9810058431314316 
Score Teste:  0.9791543544401263
Erro médio absoluto:  175.3507491543762 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Árvores de Decisão

In [ ]:
sc1_ad, sc2_ad, mae_ad = arvore_decisao(X_train, Y_train, X_test, Y_test)

Score Treino:  1.0 
Score Teste:  0.976302869785816
Erro médio absoluto:  161.66666666666666 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Random Forest

In [ ]:
sc1_rf, sc2_rf, mae_rf = floresta_randomica(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9947641354770748 
Score Teste:  0.9801284231525191
Erro médio absoluto:  146.41410256410256 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão MLP

In [ ]:
sc1_rn, sc2_rn, mae_rn = mlp(X_train, Y_train, X_test, Y_test, hls=(200, 100))

Score Treino:  4.0223999360271224e-05 
Score Teste:  -0.12504620272749237
Erro médio absoluto:  1356.2003339477942 



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, pleas

##### Regressão KNN

In [ ]:
sc1_kn, sc2_kn, mae_kn = knr(X_train, Y_train, X_test, Y_test, k=5)

Score Treino:  0.52425255620791 
Score Teste:  -0.31504530516409956
Erro médio absoluto:  1521.6350427350426 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Resultados

In [ ]:
regressores = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino = [sc1_rl, sc1_rp, sc1_ad, sc1_rf, sc1_rn, sc1_kn]
scores_teste = [sc2_rl, sc2_rp, sc2_ad, sc2_rf, sc2_rn, sc2_kn]
maes = [mae_rl, mae_rp, mae_ad, mae_rf, mae_rn, mae_kn]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores de TREINO/TESTE dos regressores na base de pessoas agrupadas (mes, ano, sexo).')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de pessoas agrupadas (mes, ano, sexo).', labels={'x': 'Regressores', 'y': 'MAE'})

##### Extrapolação dos dados

###### Criando as novas amostras (X)

In [ ]:
import numpy as np
monthNames = ['July', 'August', 'September', 'October', 'November', 'December'];
ano = ['2020']
sexo = df['sexo'].unique()

comb  = np.array(np.meshgrid(ano, monthNames, sexo)).T.reshape(-1,3)
df_novo1 = pd.DataFrame(comb, columns=['ano', 'mes', 'sexo'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(ano, monthNames, sexo)).T.reshape(-1,3)
df_novo2 = pd.DataFrame(comb, columns=['ano', 'mes', 'sexo'])

df_novo = pd.concat([df_novo1, df_novo2])
df_novo.index = [i for i in range(len(df_novo))]

df_novo_concatenado = pd.concat([df, df_novo])

LE = LabelEncoder()
df_novo_concatenado['sexo'] = LE.fit_transform(df_novo_concatenado['sexo'])
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concatenado.loc[df_novo_concatenado.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concatenado.index = [i for i in range(len(df_novo_concatenado))]
df_novo_concatenado

,ano,mes,sexo,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2017,4,0,2581.0,3939.0,1540.0,1908.0,394.0,97.0
1,2017,4,1,7223.0,12719.0,7359.0,3745.0,1204.0,411.0
2,2017,4,2,975.0,1245.0,260.0,13.0,3.0,7.0
3,2017,8,0,2307.0,3317.0,1252.0,1574.0,397.0,94.0
4,2017,8,1,6614.0,11433.0,6258.0,3547.0,1224.0,404.0
...,...,...,...,...,...,...,...,...,...
145,2020,11,2,NaN,NaN,NaN,NaN,NaN,NaN
146,2020,12,2,NaN,NaN,NaN,NaN,NaN,NaN
147,2021,1,0,NaN,NaN,NaN,NaN,NaN,NaN
148,2021,1,1,NaN,NaN,NaN,NaN,NaN,NaN


###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concatenado[:len(df)][x]
X_test = df_novo_concatenado[len(df):][x]
Y_train = df[y]

###### Random Forest

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train, Y_train)

previsoes = regressor.predict(X_test)

previsoes = pd.DataFrame(previsoes)
previsoes.columns = y

previsoes = pd.concat([df_novo, previsoes], axis=1)
for mes in meses:
  previsoes.loc[previsoes.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes

,ano,mes,sexo,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2020,7,Feminino,1510.24,2079.75,696.85,1074.05,248.00,60.85
1,2020,8,Feminino,1507.69,2069.91,689.23,1073.09,246.86,60.73
2,2020,9,Feminino,1514.62,2081.83,691.74,1081.59,247.29,61.21
3,2020,10,Feminino,1522.36,2095.87,697.49,1084.45,252.06,61.87
4,2020,11,Feminino,1524.34,2096.80,698.82,1085.08,251.47,61.43
5,2020,12,Feminino,1730.23,2499.58,837.47,1293.89,296.24,71.98
6,2020,7,Masculino,4649.35,8247.77,3980.51,2913.92,991.18,362.16
7,2020,8,Masculino,4650.29,8244.10,3976.86,2915.81,989.82,361.61
8,2020,9,Masculino,4668.01,8276.71,3992.83,2928.41,994.64,360.83
9,2020,10,Masculino,4675.20,8292.33,3999.64,2935.20,998.39,359.10


##### Plot Extrapolação

In [ ]:
def cria_data_plot(attr):
  data_plot = list()
  for sexo in df_plot.sexo.unique():
    data = df_plot[df_plot.sexo == sexo]
    data_plot.append(go.Line(name=(sexo + '- Real'), x=data.mes.values, y=data[attr].values))

  for sexo in previsoes.sexo.unique():
    data = previsoes[(previsoes.sexo == sexo) & (previsoes.ano == '2020')]
    data_plot.append(go.Line(name=(sexo + '- Previsto'), x=data.mes.values, y=data[attr].values))
  return data_plot

In [ ]:
df_novo_concatenado['sexo'] = LE.inverse_transform(df_novo_concatenado['sexo'])

df_plot = df_novo_concatenado[:len(df)][df_novo_concatenado[:len(df)]['ano'] == '2020']
df_plot = df_plot.sort_values('mes')

###### Quantidade de Acidentes

In [ ]:
data_plot = cria_data_plot('qtd_acidentes')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de acidentes por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quantidade de Pessoas

In [ ]:
data_plot = cria_data_plot('qtd_pessoas')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Pessoas por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quatidade de Ilesos

In [ ]:
data_plot = cria_data_plot('qtd_ilesos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Ilesos por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quatidade de Feridos Leves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_leves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Feridos Leves por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quatidade de Feridos Graves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_graves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Graves Leves por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




###### Quatidade de Mortos

In [ ]:
data_plot = cria_data_plot('qtd_mortos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Mortos Leves por mês em 2020 em cada sexo- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## **Base Ocorrências**
---

#### **Hipótese 1:**

----

- *Iniciei com todos os atributos de entrada e fui retirando*
- *Desde o início os resultados foram ruins*
- *Fui retirando atributos e os resultados continuaram ruins do mesmo jeito*
- *Retirei o atributo 'classificacao_acidente' e piorou mais ainda*
- *Então, coloquei ele de volta e retirei o atributo 'municipios' e, finalmente, obtive bons resultados com o* **Árvore de Decisão(68,36%)** *e* **Random Forest(80,33%)**
- *Retirei o atributo 'tipo_acidente' e o score de teste do Árvore de Decisão e Random Forest aumentaram para 90,27% e 93,61%  respectivamente.*

##### Agrupamento da base

-----

In [ ]:
x = ['ano', 'mes', 'uf', 'causa_acidente', 'classificacao_acidente']
y = ['qt_acidentes', 'qt_pessoas', 'qt_mortos', 'qt_feridos_leves', 'qt_feridos_graves', 'qt_ilesos', 'qt_ignorados', 'qt_feridos', 'qt_veiculos']

df = dfOcorrencias
df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%m'), 'uf', 'causa_acidente', 'classificacao_acidente'])
df = df.agg({'id': 'count', 'pessoas': 'sum', 'mortos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'ilesos': 'sum', 'ignorados': 'sum', 'feridos': 'sum', 'veiculos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

##### Tratamento da base

----

In [ ]:
# Label Encoder
categoricosOcorrencias = ['uf', 'causa_acidente', 'classificacao_acidente']
df_encoded = label_encoder(df.copy(), categoricosOcorrencias)

# Hold-Out
X = df_encoded[x]
Y = df_encoded[y]
X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Algoritmos de regressão

----

###### Regressão Linear:

In [ ]:
sc1_rl, sc2_rl, mae_rl = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.00868414792179301 
Score Teste:  0.007472341101122346
Erro médio absoluto:  7.676867024083001 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



###### Regressão Polinomial:

In [ ]:
sc1_rp, sc2_rp, mae_rp = regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.09132815890509037 
Score Teste:  0.08814897914473163
Erro médio absoluto:  7.492008871793963 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



###### Regressão com Árvore de Decisão:

In [ ]:
sc1_ad, sc2_ad, mae_ad = arvore_decisao(X_train, Y_train, X_test, Y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



Score Treino:  1.0 
Score Teste:  0.9031143248945399
Erro médio absoluto:  2.7703392903392903 



###### Regressão com Random Forest:

In [ ]:
sc1_rf, sc2_rf, mae_rf = floresta_randomica(X_train, Y_train, X_test, Y_test, n=20)

Score Treino:  0.9904472799471572 
Score Teste:  0.93279787014591
Erro médio absoluto:  2.3151810411810416 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



###### Regressão com MLP

In [ ]:
sc1_rm, sc2_rm, mae_rm = mlp(X_train, Y_train, X_test, Y_test)

Score Treino:  0.005007016833580339 
Score Teste:  0.004059248252637433
Erro médio absoluto:  7.709140712447207 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



###### Regressão com K vizinhos

In [ ]:
sc1_rk, sc2_rk, mae_rk = knr(X_train, Y_train, X_test, Y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



Score Treino:  0.7439914978985005 
Score Teste:  0.2794166049500784
Erro médio absoluto:  6.008728308728309 



##### Resultados

In [ ]:
regressores = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino = [sc1_rl, sc1_rp, sc1_ad, sc1_rf, sc1_rm, sc1_rk]
scores_teste = [sc2_rl, sc2_rp, sc2_ad, sc2_rf, sc2_rm, sc2_rk]
maes = [mae_rl, mae_rp, mae_ad, mae_rf, mae_rm, mae_rk]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores do TREINO/TESTE dos regressores na base de ocorências.')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de ocorrências.')

##### Extrapolação

###### Criando as novas amostras (X)

In [ ]:
import numpy as np
monthNames = ['July', 'August', 'September', 'October', 'November', 'December'];
ano = ['2020']
uf = df['uf'].unique()
causa_acidente = df['causa_acidente'].unique()
classificacao_acidente = df['classificacao_acidente'].unique()

comb  = np.array(np.meshgrid(ano, monthNames, uf, causa_acidente, classificacao_acidente)).T.reshape(-1,5)
df_novo1 = pd.DataFrame(comb, columns=['ano', 'mes', 'uf', 'causa_acidente', 'classificacao_acidente'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(ano, monthNames, uf, causa_acidente, classificacao_acidente)).T.reshape(-1,5)
df_novo2 = pd.DataFrame(comb, columns=['ano', 'mes', 'uf', 'causa_acidente', 'classificacao_acidente'])

df_novo = pd.concat([df_novo1, df_novo2])
df_novo.index = [i for i in range(len(df_novo))]
# df
df_novo_concatenado = pd.concat([df, df_novo])
# df_novo_concatenado

LE_uf = LabelEncoder()
LE_causa = LabelEncoder()
LE_classificacao = LabelEncoder()
df_novo_concatenado['uf'] = LE_uf.fit_transform(df_novo_concatenado['uf'])
df_novo_concatenado['causa_acidente'] = LE_causa.fit_transform(df_novo_concatenado['causa_acidente'])
df_novo_concatenado['classificacao_acidente'] = LE_classificacao.fit_transform(df_novo_concatenado['classificacao_acidente'])

meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concatenado.loc[df_novo_concatenado.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concatenado.index = [i for i in range(len(df_novo_concatenado))]
# df_novo_concatenado

###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concatenado[:len(df)][x]
X_test = df_novo_concatenado[len(df):][x]
Y_train = df[y]

###### Random Forest

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train, Y_train)

previsoes = regressor.predict(X_test)

previsoes = pd.DataFrame(previsoes)
previsoes.columns = y

previsoes = pd.concat([df_novo, previsoes], axis=1)
for mes in meses:
  previsoes.loc[previsoes.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes

,ano,mes,uf,causa_acidente,classificacao_acidente,qt_acidentes,qt_pessoas,qt_mortos,qt_feridos_leves,qt_feridos_graves,qt_ilesos,qt_ignorados,qt_feridos,qt_veiculos
0,2020,7,AC,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,1.02,2.62,0.00,1.45,0.69,0.48,0.00,2.14,1.02
1,2020,8,AC,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,1.03,2.51,0.00,1.37,0.69,0.45,0.00,2.06,1.04
2,2020,9,AC,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,1.01,2.37,0.00,1.08,0.86,0.42,0.01,1.94,1.01
3,2020,10,AC,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,1.11,2.53,0.00,1.40,0.70,0.40,0.03,2.10,1.11
4,2020,11,AC,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,1.21,2.75,0.00,1.65,0.61,0.42,0.07,2.26,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13603,2021,1,RS,Ingestão de álcool e/ou substâncias psicoativa...,Com Vítimas Fatais,1.35,5.14,1.75,0.26,0.20,1.62,1.31,0.46,1.62
13604,2021,1,SC,Ingestão de álcool e/ou substâncias psicoativa...,Com Vítimas Fatais,1.37,5.26,1.77,0.26,0.24,1.66,1.33,0.50,1.73
13605,2021,1,SE,Ingestão de álcool e/ou substâncias psicoativa...,Com Vítimas Fatais,1.37,5.24,1.77,0.26,0.24,1.64,1.33,0.50,1.73
13606,2021,1,SP,Ingestão de álcool e/ou substâncias psicoativa...,Com Vítimas Fatais,1.44,5.76,1.87,0.55,0.57,1.47,1.30,1.12,1.90


#### **Hipótese 2:**
---

##### Agrupamento da base Pessoas

In [ ]:
x = ['ano', 'mes', 'tipo_veiculo']
y = ['qtd_acidentes', 'qtd_pessoas', 'qtd_ilesos', 'qtd_feridos_leves', 'qtd_feridos_graves', 'qtd_mortos']

df = dfPessoas
df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%B'), 'tipo_veiculo'])
df = df.agg({'id': 'nunique', 'pesid': 'nunique', 'ilesos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'mortos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

#df = df[df['tipo_veiculo'] == 'Motocicleta']
#df = df.reset_index()

##### Pré-Processamento

In [ ]:
# Label Encoder
df_pessoas_tratado = label_encoder(df.copy(), ['mes', 'tipo_veiculo'])

# Divisão da base
X = df_pessoas_tratado[x]
Y = df_pessoas_tratado[y]

# Standart Scaler
# X, Y = standard_scaler(X, Y)

X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Regressão Linear

In [ ]:
sc1_rl, sc2_rl, mae_rl = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.190375238478753 
Score Teste:  0.21785405198349914
Erro médio absoluto:  349.66194055447977 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão Polinomial

In [ ]:
sc1_rp, sc2_rp, mae_rp = regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.4065775647842881 
Score Teste:  0.44126128568124817
Erro médio absoluto:  288.28534474316984 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Árvores de Decisão

In [ ]:
sc1_ad, sc2_ad, mae_ad = arvore_decisao(X_train, Y_train, X_test, Y_test)

Score Treino:  1.0 
Score Teste:  0.9630353470858573
Erro médio absoluto:  40.8984375 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Random Forest

In [ ]:
sc1_rf, sc2_rf, mae_rf = floresta_randomica(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9945333195492919 
Score Teste:  0.9739502832440489
Erro médio absoluto:  34.509114583333336 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão MLP

In [ ]:
sc1_rn, sc2_rn, mae_rn = mlp(X_train, Y_train, X_test, Y_test, hls=(200, 100))

Score Treino:  0.15608457248064683 
Score Teste:  0.18247804976030527
Erro médio absoluto:  293.7153789459597 



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, pleas

##### Regressão KNN

In [ ]:
sc1_kn, sc2_kn, mae_kn = knr(X_train, Y_train, X_test, Y_test, k=5)

Score Treino:  0.8081750792488782 
Score Teste:  0.6900325666961666
Erro médio absoluto:  151.77031250000002 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Resultados

In [ ]:
regressores = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino = [sc1_rl, sc1_rp, sc1_ad, sc1_rf, sc1_rn, sc1_kn]
scores_teste = [sc2_rl, sc2_rp, sc2_ad, sc2_rf, sc2_rn, sc2_kn]
maes = [mae_rl, mae_rp, mae_ad, mae_rf, mae_rn, mae_kn]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores de TREINO/TESTE dos regressores na base de pessoas agrupadas (mes, ano, tipo_veiculo).')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de pessoas agrupadas (mes, ano, sexo).')

##### Extrapolação dos dados

###### Criando as novas amostras (X)

In [ ]:
import numpy as np
monthNames = ['July', 'August', 'September', 'October', 'November', 'December'];
ano = ['2020']
tipo_veiculo = df['tipo_veiculo'].unique()

comb  = np.array(np.meshgrid(ano, monthNames, tipo_veiculo)).T.reshape(-1,3)
df_novo1 = pd.DataFrame(comb, columns=['ano', 'mes', 'tipo_veiculo'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(ano, monthNames, tipo_veiculo)).T.reshape(-1,3)
df_novo2 = pd.DataFrame(comb, columns=['ano', 'mes', 'tipo_veiculo'])

df_novo = pd.concat([df_novo1, df_novo2])
df_novo.index = [i for i in range(len(df_novo))]

df_novo_concatenado = pd.concat([df, df_novo])

LE = LabelEncoder()
df_novo_concatenado['tipo_veiculo'] = LE.fit_transform(df_novo_concatenado['tipo_veiculo'])
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concatenado.loc[df_novo_concatenado.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concatenado.index = [i for i in range(len(df_novo_concatenado))]
df_novo_concatenado

,ano,mes,tipo_veiculo,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2017,4,0,4453.0,8474.0,4852.0,2415.0,565.0,192.0
1,2017,4,1,171.0,188.0,8.0,108.0,42.0,15.0
2,2017,4,2,1090.0,1643.0,1019.0,400.0,111.0,38.0
3,2017,4,3,998.0,1261.0,896.0,189.0,53.0,33.0
4,2017,4,4,1156.0,1352.0,1053.0,184.0,45.0,27.0
...,...,...,...,...,...,...,...,...,...
1022,2021,1,24,NaN,NaN,NaN,NaN,NaN,NaN
1023,2021,1,15,NaN,NaN,NaN,NaN,NaN,NaN
1024,2021,1,18,NaN,NaN,NaN,NaN,NaN,NaN
1025,2021,1,22,NaN,NaN,NaN,NaN,NaN,NaN


###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concatenado[:len(df)][x]
X_test = df_novo_concatenado[len(df):][x]
Y_train = df[y]

###### Random Forest

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train, Y_train)

previsoes = regressor.predict(X_test)

previsoes = pd.DataFrame(previsoes)
previsoes.columns = y

previsoes = pd.concat([df_novo, previsoes], axis=1)
for mes in meses:
  previsoes.loc[previsoes.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes

,ano,mes,tipo_veiculo,qtd_acidentes,qtd_pessoas,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos
0,2020,7,Automóvel,2532.64,4618.48,2297.29,1441.63,371.00,154.52
1,2020,8,Automóvel,2538.48,4621.28,2296.85,1443.90,371.72,154.21
2,2020,9,Automóvel,2539.80,4625.02,2301.28,1446.29,370.21,154.39
3,2020,10,Automóvel,2541.12,4627.34,2302.57,1447.42,371.18,154.47
4,2020,11,Automóvel,2551.88,4658.76,2319.93,1455.98,373.76,156.12
...,...,...,...,...,...,...,...,...,...
170,2021,1,Ônibus,139.38,632.67,366.41,196.98,43.64,11.39
171,2021,1,Quadriciclo,1.03,1.04,0.59,0.02,0.01,0.04
172,2021,1,Trator de esteira,2.32,2.45,0.82,0.95,0.21,0.02
173,2021,1,Triciclo,1.63,1.79,0.77,0.76,0.06,0.18


##### Plot Extrapolação

In [ ]:
def cria_data_plot(attr):
  data_plot = list()
  for tipo_veiculo in df_plot.tipo_veiculo.unique():
    data = df_plot[df_plot.tipo_veiculo == tipo_veiculo]
    data_plot.append(go.Line(name=(tipo_veiculo + '- Real'), x=data.mes.values, y=data[attr].values))

  for tipo_veiculo in previsoes.tipo_veiculo.unique():
    data = previsoes[(previsoes.tipo_veiculo == tipo_veiculo) & (previsoes.ano == '2020')]
    data_plot.append(go.Line(name=(tipo_veiculo + '- Previsto'), x=data.mes.values, y=data[attr].values))
  return data_plot

In [ ]:
df_novo_concatenado['tipo_veiculo'] = LE.inverse_transform(df_novo_concatenado['tipo_veiculo'])

df_plot = df_novo_concatenado[:len(df)][df_novo_concatenado[:len(df)]['ano'] == '2020']
df_plot = df_plot.sort_values('mes')

##### Quantidade de Acidentes

In [ ]:
data_plot = cria_data_plot('qtd_acidentes')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quantidade de Pessoas

In [ ]:
data_plot = cria_data_plot('qtd_pessoas')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Pessoas envolvidas em acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Ilesos

In [ ]:
data_plot = cria_data_plot('qtd_ilesos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Ilesos envolvidos em acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Feridos Leves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_leves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Feridos Leves envolvidos em acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Feridos Graves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_graves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de vítimas em estado grave envolvidos em acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Mortos

In [ ]:
data_plot = cria_data_plot('qtd_mortos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Mortos envolvidos em acidentes, por mês, em 2020 por tipo de veículo - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




#### **Hipótese 3:**
---


##### Agrupamento da base Ocorrências

In [ ]:
x = ['ano', 'mes', 'causa_acidente']
y = ['qtd_acidentes', 'qtd_ilesos', 'qtd_feridos_leves', 'qtd_feridos_graves', 'qtd_mortos', 'qtd_feridos', 'qtd_veiculos']

df = dfOcorrencias
df = df.groupby([df['data_inversa'].dt.strftime('%Y'), df['data_inversa'].dt.strftime('%B'), 'causa_acidente'])
df = df.agg({'id': 'nunique', 'ilesos': 'sum', 'feridos_leves': 'sum', 'feridos_graves': 'sum', 'mortos': 'sum', 'feridos': 'sum', 'veiculos': 'sum'})
df.index.names = x
df.columns = y
df = df.reset_index()

#df = df[df['causa_acidente'] == 'Animais na Pista']
#df = df.reset_index()

##### Pré-Processamento

In [ ]:
# Label Encoder
df_ocorrencias_tratado = label_encoder(df.copy(), ['mes', 'causa_acidente'])

# Divisão da base
X = df_ocorrencias_tratado[x]
Y = df_ocorrencias_tratado[y]

# Standart Scaler
# X, Y = standard_scaler(X, Y)

X_train, X_test, Y_train, Y_test = hold_out(X, Y)

##### Regressão Linear

In [ ]:
sc1_rl, sc2_rl, mae_rl = regressao_linear(X_train, Y_train, X_test, Y_test)

Score Treino:  0.012139609685341118 
Score Teste:  0.0017737006488780863
Erro médio absoluto:  219.022049395509 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão Polinomial

In [ ]:
sc1_rp, sc2_rp, mae_rp = regressao_polinomial(X_train, Y_train, X_test, Y_test)

Score Treino:  0.09251027449173825 
Score Teste:  0.01790589734399044
Erro médio absoluto:  224.59950523540502 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Árvores de Decisão

In [ ]:
sc1_ad, sc2_ad, mae_ad = arvore_decisao(X_train, Y_train, X_test, Y_test)

Score Treino:  1.0 
Score Teste:  0.9730457688829115
Erro médio absoluto:  31.161490683229808 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão por Random Forest

In [ ]:
sc1_rf, sc2_rf, mae_rf = floresta_randomica(X_train, Y_train, X_test, Y_test)

Score Treino:  0.9958899250365785 
Score Teste:  0.9755635984630532
Erro médio absoluto:  27.907978977544197 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão MLP

In [ ]:
sc1_rn, sc2_rn, mae_rn = mlp(X_train, Y_train, X_test, Y_test, hls=(200, 100))

Score Treino:  -0.00013295382509632576 
Score Teste:  -0.0014519262413858387
Erro médio absoluto:  220.68860543432515 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Regressão KNN

In [ ]:
sc1_kn, sc2_kn, mae_kn = knr(X_train, Y_train, X_test, Y_test, k=5)

Score Treino:  0.791031266646652 
Score Teste:  0.5643560618747276
Erro médio absoluto:  145.47214524605832 



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



##### Resultados

In [ ]:
regressores = ['Linear', 'Polinomial', 'Decision Tree', 'Random Forest', 'MLP', 'KNN']
scores_treino = [sc1_rl, sc1_rp, sc1_ad, sc1_rf, sc1_rn, sc1_kn]
scores_teste = [sc2_rl, sc2_rp, sc2_ad, sc2_rf, sc2_rn, sc2_kn]
maes = [mae_rl, mae_rp, mae_ad, mae_rf, mae_rn, mae_kn]

In [ ]:
plot_scores_bar(regressores, scores_treino, scores_teste, 'Scores de TREINO/TESTE dos regressores na base de pessoas agrupadas (mes, ano, tipo_veiculo).')

In [ ]:
px.bar(x=regressores, y=maes, color=maes , title='ERRO MÉDIO ABSOLUTO dos regressores na base de pessoas agrupadas (mes, ano, sexo).')

##### Extrapolação dos dados

###### Criando as novas amostras (X)

In [ ]:
import numpy as np
monthNames = ['July', 'August', 'September', 'October', 'November', 'December'];
ano = ['2020']
causa_acidente = df['causa_acidente'].unique()

comb  = np.array(np.meshgrid(ano, monthNames, causa_acidente)).T.reshape(-1,3)
df_novo1 = pd.DataFrame(comb, columns=['ano', 'mes', 'causa_acidente'])

monthNames = ["January"];
ano = ['2021']

comb  = np.array(np.meshgrid(ano, monthNames, causa_acidente)).T.reshape(-1,3)
df_novo2 = pd.DataFrame(comb, columns=['ano', 'mes', 'causa_acidente'])

df_novo = pd.concat([df_novo1, df_novo2])
df_novo.index = [i for i in range(len(df_novo))]

df_novo_concatenado = pd.concat([df, df_novo])

LE = LabelEncoder()
df_novo_concatenado['causa_acidente'] = LE.fit_transform(df_novo_concatenado['causa_acidente'])
meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for mes in meses:
  df_novo_concatenado.loc[df_novo_concatenado.mes == mes, 'mes'] = meses.index(mes) + 1
df_novo_concatenado.index = [i for i in range(len(df_novo_concatenado))]
df_novo_concatenado

,ano,mes,causa_acidente,qtd_acidentes,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos,qtd_feridos,qtd_veiculos
0,2017,4,1,258.0,242.0,134.0,36.0,14.0,170.0,285.0
1,2017,4,2,120.0,121.0,124.0,27.0,1.0,151.0,140.0
2,2017,4,3,42.0,47.0,17.0,3.0,0.0,20.0,60.0
3,2017,4,4,322.0,295.0,217.0,58.0,31.0,275.0,421.0
4,2017,4,5,382.0,386.0,261.0,67.0,7.0,328.0,503.0
...,...,...,...,...,...,...,...,...,...,...
1157,2021,1,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1158,2021,1,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1159,2021,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,2021,1,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###### Dividindo a base em treino e teste

In [ ]:
X_train = df_novo_concatenado[:len(df)][x]
X_test = df_novo_concatenado[len(df):][x]
Y_train = df[y]

###### Random Forest

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train, Y_train)

previsoes = regressor.predict(X_test)

previsoes = pd.DataFrame(previsoes)
previsoes.columns = y

previsoes = pd.concat([df_novo, previsoes], axis=1)
for mes in meses:
  previsoes.loc[previsoes.mes == mes, 'mes'] = meses.index(mes) + 1
previsoes

,ano,mes,causa_acidente,qtd_acidentes,qtd_ilesos,qtd_feridos_leves,qtd_feridos_graves,qtd_mortos,qtd_feridos,qtd_veiculos
0,2020,7,Animais na Pista,114.26,70.45,80.52,27.72,5.94,108.24,133.47
1,2020,8,Animais na Pista,114.83,70.96,80.53,27.89,5.93,108.42,134.09
2,2020,9,Animais na Pista,115.98,70.76,88.11,28.26,5.39,116.37,135.75
3,2020,10,Animais na Pista,116.26,67.77,85.34,29.43,5.33,114.77,134.11
4,2020,11,Animais na Pista,114.09,69.98,84.28,26.94,5.93,111.22,131.88
...,...,...,...,...,...,...,...,...,...,...
163,2021,1,Ultrapassagem Indevida,96.14,115.46,111.71,54.84,19.51,166.55,192.63
164,2021,1,Velocidade Incompatível,556.27,423.50,588.66,154.50,56.41,743.16,763.06
165,2021,1,Agressão Externa,38.48,31.52,30.88,6.49,3.10,37.37,49.67
166,2021,1,Desobediência às normas de trânsito pelo pedestre,26.03,24.80,13.13,7.68,9.36,20.81,31.46


##### Plot Extrapolação

In [ ]:
def cria_data_plot(attr):
  data_plot = list()
  for causa_acidente in df_plot.causa_acidente.unique():
    data = df_plot[df_plot.causa_acidente == causa_acidente]
    data_plot.append(go.Line(name=(causa_acidente + '- Real'), x=data.mes.values, y=data[attr].values))

  for causa_acidente in previsoes.causa_acidente.unique():
    data = previsoes[(previsoes.causa_acidente == causa_acidente) & (previsoes.ano == '2020')]
    data_plot.append(go.Line(name=(causa_acidente + '- Previsto'), x=data.mes.values, y=data[attr].values))
  return data_plot

In [ ]:
df_novo_concatenado['causa_acidente'] = LE.inverse_transform(df_novo_concatenado['causa_acidente'])

df_plot = df_novo_concatenado[:len(df)][df_novo_concatenado[:len(df)]['ano'] == '2020']
df_plot = df_plot.sort_values('mes')

##### Quantidade de Acidentes

In [ ]:
data_plot = cria_data_plot('qtd_acidentes')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de acidentes, por mês, em 2020 em cada causa de acidente- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Ilesos

In [ ]:
data_plot = cria_data_plot('qtd_ilesos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Ilesos envolvidos em acidentes, por mês, em 2020 em cada causa de acidente - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Feridos Leves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_leves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Feridos Leves envolvidos em acidentes, por mês, em 2020 em cada causa de acidente- Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Feridos Graves

In [ ]:
data_plot = cria_data_plot('qtd_feridos_graves')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de vítimas em estado grave envolvidos em acidentes, por mês, em 2020 em cada causa de acidente - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Mortos

In [ ]:
data_plot = cria_data_plot('qtd_mortos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Mortos envolvidos em acidentes, por mês, em 2020 em cada causa de acidente - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Feridos

In [ ]:
data_plot = cria_data_plot('qtd_feridos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Feridos envolvidos em acidentes, por mês, em 2020 em cada causa de acidente - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Quatidade de Veículos

In [ ]:
data_plot = cria_data_plot('qtd_veiculos')
fig = go.Figure(data=data_plot)
fig.update_layout(title_text='Quantidade de Veículos envolvidos em acidentes, por mês, em 2020 por cada causa de acidente - Dados reais e previstos')
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## **Conclusão**

----

- Os algoritmos de Regressão que geraram melhores resultados, no geral, foram: Árvore de Decisão, Random Forest e Regressão Polinomial. Para algumas hipóteses não foi plotado o gráfico que exibe a curva de extrapolação para o ano de 2020 pois, haviam numerosas classes em dados atributos, o que dificultou a exibição.